*****
## WFS Layer Extract
*****
Author: Mackenzie Rock

Date: June 3, 2025

Goal: The goal of this Jupyter notebook is to determine a foundation for extracting the relevant WFS layers identified and storing them in a suitable format for uploading to the postgreSQL database. In this section I will test:
- The code to extract
- I will visualize the data to ensure I have capture it correctly
- I will test several samples overtime
- Transformation into suitable format for load

### Extraction Code

To be extracted from WFS:
- Fire Danger (PUBLIC:FDR_CURRENT_SHP)
- Fire Perimeter Estimate (PUBLIC:M3_POLYGONS_CURRENT)
- Fire M3 Hotspots (PUBLIC:HOTSPOTS_LAST24HRS)
- Season-to-date Hotspots (TBD)
- Active Fires (PUBLIC:ACTIVEFIRES_CURRENT)
- Forecast Weather Stations (PUBLIC:FIREWX_STNS & PUBLIC:FIREWX_STNS_CURRENT)
- Reporting Weather Stations (PUBLIC:FIREWX_SCRIBE & PUBLIC:FIREWX_SCRIBE_FCST)
- Fire History (PUBLIC:REPORTEDFIRES_2024 & PUBLIC:REPORTEDFIRES_YTD)
- Check on what PUBLIC:BASEMAP_INSIDE_BNDRY & PUBLIC:BASEMAP_LAND are

In [7]:
import geopandas as gpd
import folium
from shapely.geometry import mapping
import os
import requests
from io import BytesIO
import pandas as pd

# WFS endpoint
WFS_URL = "https://cwfis.cfs.nrcan.gc.ca/geoserver/public/ows"
OUTPUT_DIR = "./wfs_layers"
MAP_OUTPUT_DIR = "wfs_maps"

# Ensure output dirs exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MAP_OUTPUT_DIR, exist_ok=True)



def fetch_and_visualize_wfs_layer(layer_name: str, label: str = None, date: str = None, output_format: str = "GeoJSON"):
    """
    Fetches a layer from the CWFIS WFS endpoint, reprojects to EPSG:4326,
    saves to file, and visualizes using Folium.
    """
    # Build WFS URL
    params = {
        "service": "WFS",
        "version": "1.0.0",
        "request": "GetFeature",
        "typename": layer_name,
        "outputFormat": "application/json"
    }
    query_url = f"{WFS_URL}?{'&'.join([f'{k}={v}' for k, v in params.items()])}"

    print(f"Fetching {layer_name} from WFS...")


    # Read GeoDataFrame
    response = requests.get(query_url)
    response.raise_for_status()  # Fail early if bad response
    gdf = gpd.read_file(BytesIO(response.content))

    # Reproject if needed
    if gdf.crs and gdf.crs.to_string() != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")


    json_safe_gdf = gdf.copy()
    for col in json_safe_gdf.columns:
        if pd.api.types.is_datetime64_any_dtype(json_safe_gdf[col]) or isinstance(json_safe_gdf[col].iloc[0], pd.Timestamp):
            print(f"Converting datetime column '{col}' to string")
            json_safe_gdf[col] = json_safe_gdf[col].astype(str)
        elif isinstance(json_safe_gdf[col].iloc[0], (list, dict)):
            json_safe_gdf[col] = json_safe_gdf[col].astype(str)


    # Save GeoJSON
    label_safe = label.replace(" ", "_").lower() if label else layer_name.replace(":", "_").lower()
    filename = f"{label_safe}_{date or 'latest'}.{output_format.lower()}"
    filepath = os.path.join(OUTPUT_DIR, filename)
    json_safe_gdf.to_file(filepath, driver="GeoJSON")

    # Create map centered on centroid of geometry
    centroid = json_safe_gdf.geometry.unary_union.centroid
    fmap = folium.Map(location=[centroid.y, centroid.x], zoom_start=5)

    # Add layer as external file read
    with open(filepath, "r", encoding="utf-8") as f:
        folium.GeoJson(data=f.read(), name=label or layer_name).add_to(fmap)

    # Save map
    html_map_path = os.path.join(MAP_OUTPUT_DIR, f"{label_safe}_{date or 'latest'}.html")
    fmap.save(html_map_path)


In [8]:
#from wfs_utils import fetch_and_visualize_wfs_layer

map_output_dir = "wfs_maps"
layer_names = {
    "Fire_Danger": "public:fdr_current_shp",
    "Fire_Perimeter_Estimate": "public:m3_polygons_current",
    "M3_Hotspots": "public:hotspots_last24hrs",
    "Active_Fires": "public:activefires_current",
    "Forecast_Weather_Stations": "public:firewx_stns",
    "Forecast_Weather_Stations_Current": "public:firewx_stns_current",
    "Reporting_Weather_Stations": "public:firewx_scribe",
    "Reporting_Weather_Stations_Forecast": "public:firewx_scribe_fcst",
    "Fire_History_YTD": "public:reportedfires_ytd",
    "Fire_History_2024": "public:reportedfires_2024"
}
dates = ["2024-01-01"]


for date in dates:
    for label, type in layer_names.items():

        print(f"Attempting to process layer: {label}")
        fetch_and_visualize_wfs_layer(type, label, date)





Attempting to process layer: Fire_Danger
Fetching public:fdr_current_shp from WFS...


/tmp/ipykernel_36544/2130008081.py:64: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = json_safe_gdf.geometry.unary_union.centroid


Attempting to process layer: Fire_Perimeter_Estimate
Fetching public:m3_polygons_current from WFS...
Converting datetime column 'firstdate' to string
Converting datetime column 'lastdate' to string


/tmp/ipykernel_36544/2130008081.py:64: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = json_safe_gdf.geometry.unary_union.centroid


Attempting to process layer: M3_Hotspots
Fetching public:hotspots_last24hrs from WFS...
Converting datetime column 'rep_date' to string


...

/tmp/ipykernel_36544/2130008081.py:64: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = json_safe_gdf.geometry.unary_union.centroid


Converting datetime column 'startdate' to string
Attempting to process layer: Fire_History_2024
Fetching public:reportedfires_2024 from WFS...


/tmp/ipykernel_36544/2130008081.py:64: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = json_safe_gdf.geometry.unary_union.centroid


Converting datetime column 'startdate' to string


/tmp/ipykernel_36544/2130008081.py:64: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = json_safe_gdf.geometry.unary_union.centroid


### Format for Stoage
- I will store the data in GPKG for storage reasons. This data will be extracted and transformed to GeoJSON for visualization

In [3]:
import geopandas as gpd
import folium
from shapely.geometry import mapping
import os
import requests
from io import BytesIO
import pandas as pd

# WFS endpoint
WFS_URL = "https://cwfis.cfs.nrcan.gc.ca/geoserver/public/ows"
OUTPUT_DIR = "./wfs_layers"
MAP_OUTPUT_DIR = "wfs_maps"

# Ensure output dirs exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MAP_OUTPUT_DIR, exist_ok=True)


def fetch_and_visualize_wfs_layer(layer_name: str, label: str = None, date: str = None, output_format: str = ""):
    """
    Fetches a layer from the CWFIS WFS endpoint, reprojects to EPSG:4326,
    saves to file, and visualizes using Folium.
    """
    # Build WFS URL
    params = {
        "service": "WFS",
        "version": "1.0.0",
        "request": "GetFeature",
        "typename": layer_name,
        "outputFormat": "application/json"
    }
    query_url = f"{WFS_URL}?{'&'.join([f'{k}={v}' for k, v in params.items()])}"

    print(f"Fetching {layer_name} from WFS...")

    # Read GeoDataFrame
    response = requests.get(query_url)
    response.raise_for_status()  # Fail early if bad response
    gdf = gpd.read_file(BytesIO(response.content))

    # Reproject if needed
    if gdf.crs and gdf.crs.to_string() != "EPSG:4326":
        gdf = gdf.to_crs("EPSG:4326")

    json_safe_gdf = gdf.copy()
    for col in json_safe_gdf.columns:
        if pd.api.types.is_datetime64_any_dtype(json_safe_gdf[col]) or isinstance(json_safe_gdf[col].iloc[0],
                                                                                  pd.Timestamp):
            print(f"Converting datetime column '{col}' to string")
            json_safe_gdf[col] = json_safe_gdf[col].astype(str)
        elif isinstance(json_safe_gdf[col].iloc[0], (list, dict)):
            json_safe_gdf[col] = json_safe_gdf[col].astype(str)

    # Save gpkg
    label_safe = label.replace(" ", "_").lower() if label else layer_name.replace(":", "_").lower()
    filename = f"{label_safe}_{date or 'latest'}.{output_format.lower()}"
    filepath = os.path.join(OUTPUT_DIR, filename)
    gdf.to_file(f"{filepath}gpkg", layer="fire_perimeter", driver="GPKG")


#from wfs_utils import fetch_and_visualize_wfs_layer

map_output_dir = "wfs_maps"
layer_names = {
    "Fire_Danger": "public:fdr_current_shp",
    "Fire_Perimeter_Estimate": "public:m3_polygons_current",
    "M3_Hotspots": "public:hotspots_last24hrs",
    "Active_Fires": "public:activefires_current",
    "Forecast_Weather_Stations": "public:firewx_stns",
    "Forecast_Weather_Stations_Current": "public:firewx_stns_current",
    "Reporting_Weather_Stations": "public:firewx_scribe",
    "Reporting_Weather_Stations_Forecast": "public:firewx_scribe_fcst",
    "Fire_History_YTD": "public:reportedfires_ytd",
    "Fire_History_2024": "public:reportedfires_2024"
}
dates = ["2024-01-01"]

for date in dates:
    for label, type in layer_names.items():
        print(f"Attempting to process layer: {label}")
        fetch_and_visualize_wfs_layer(type, label, date)




Attempting to process layer: Fire_Danger
Fetching public:fdr_current_shp from WFS...
Attempting to process layer: Fire_Perimeter_Estimate
Fetching public:m3_polygons_current from WFS...
Converting datetime column 'firstdate' to string
Converting datetime column 'lastdate' to string
Attempting to process layer: M3_Hotspots
Fetching public:hotspots_last24hrs from WFS...
Converting datetime column 'rep_date' to string
Attempting to process layer: Active_Fires
Fetching public:activefires_current from WFS...
Converting datetime column 'startdate' to string
Attempting to process layer: Forecast_Weather_Stations
Fetching public:firewx_stns from WFS...
Converting datetime column 'rep_date' to string
Attempting to process layer: Forecast_Weather_Stations_Current
Fetching public:firewx_stns_current from WFS...
Converting datetime column 'rep_date' to string
Attempting to process layer: Reporting_Weather_Stations
Fetching public:firewx_scribe from WFS...
Converting datetime column 'rep_date' to s

#### Assessing the reporting date within forecast weather stations

In [10]:
import json
from collections import defaultdict
from datetime import datetime

# Load your GeoJSON data
with open("wfs_layers/forecast_weather_stations_2025-06-01.geojson", "r") as f:
    data = json.load(f)

# Store min/max rep_date per station id
station_dates = defaultdict(lambda: {"min": None, "max": None})

for feature in data["features"]:
    station_id = feature["properties"]["id"]
    rep_date_str = feature["properties"]["rep_date"]
    rep_date = datetime.fromisoformat(rep_date_str.replace("Z", "+00:00"))

    if (station_dates[station_id]["min"] is None) or (rep_date < station_dates[station_id]["min"]):
        station_dates[station_id]["min"] = rep_date
    if (station_dates[station_id]["max"] is None) or (rep_date > station_dates[station_id]["max"]):
        station_dates[station_id]["max"] = rep_date

# Print summary
for station_id, dates in station_dates.items():
    print(f"{station_id}: from {dates['min']} to {dates['max']}")

firewx_stns.8400104: from 2024-10-03 12:00:00+00:00 to 2025-04-05 12:00:00+00:00
firewx_stns.8400801: from 2024-10-22 12:00:00+00:00 to 2025-04-05 12:00:00+00:00
firewx_stns.700007: from 2024-06-18 12:00:00+00:00 to 2025-04-04 12:00:00+00:00
firewx_stns.700017: from 2024-06-18 12:00:00+00:00 to 2025-04-04 12:00:00+00:00
firewx_stns.700019: from 2024-06-18 12:00:00+00:00 to 2025-04-04 12:00:00+00:00
firewx_stns.700023: from 2024-06-18 12:00:00+00:00 to 2025-04-04 12:00:00+00:00
firewx_stns.700026: from 2024-06-18 12:00:00+00:00 to 2025-04-04 12:00:00+00:00
firewx_stns.700028: from 2024-06-18 12:00:00+00:00 to 2025-04-04 12:00:00+00:00
firewx_stns.700029: from 2024-06-18 12:00:00+00:00 to 2025-04-04 12:00:00+00:00
firewx_stns.700031: from 2024-06-18 12:00:00+00:00 to 2025-04-04 12:00:00+00:00
firewx_stns.700032: from 2024-06-18 12:00:00+00:00 to 2025-04-04 12:00:00+00:00
firewx_stns.700033: from 2024-06-18 12:00:00+00:00 to 2025-04-04 12:00:00+00:00
firewx_stns.700034: from 2024-06-18 12

In [11]:
import json
from collections import defaultdict
from datetime import datetime

# Load your GeoJSON data
with open("wfs_layers/forecast_weather_stations_current_2025-06-01.geojson", "r") as f:
    data = json.load(f)

# Store min/max rep_date per station id
station_dates = defaultdict(lambda: {"min": None, "max": None})

for feature in data["features"]:
    station_id = feature["properties"]["id"]
    rep_date_str = feature["properties"]["rep_date"]
    rep_date = datetime.fromisoformat(rep_date_str.replace("Z", "+00:00"))

    if (station_dates[station_id]["min"] is None) or (rep_date < station_dates[station_id]["min"]):
        station_dates[station_id]["min"] = rep_date
    if (station_dates[station_id]["max"] is None) or (rep_date > station_dates[station_id]["max"]):
        station_dates[station_id]["max"] = rep_date

# Print summary
for station_id, dates in station_dates.items():
    print(f"{station_id}: from {dates['min']} to {dates['max']}")

firewx_stns_current.fid--4699bdee_1973debb200_-48ae: from 2025-06-04 12:00:00+00:00 to 2025-06-04 12:00:00+00:00
firewx_stns_current.fid--4699bdee_1973debb200_-48ad: from 2025-06-04 12:00:00+00:00 to 2025-06-04 12:00:00+00:00
firewx_stns_current.fid--4699bdee_1973debb200_-48ac: from 2025-06-04 12:00:00+00:00 to 2025-06-04 12:00:00+00:00
firewx_stns_current.fid--4699bdee_1973debb200_-48ab: from 2025-06-04 12:00:00+00:00 to 2025-06-04 12:00:00+00:00
firewx_stns_current.fid--4699bdee_1973debb200_-48aa: from 2025-06-04 12:00:00+00:00 to 2025-06-04 12:00:00+00:00
firewx_stns_current.fid--4699bdee_1973debb200_-48a9: from 2025-06-04 12:00:00+00:00 to 2025-06-04 12:00:00+00:00
firewx_stns_current.fid--4699bdee_1973debb200_-48a8: from 2025-06-04 12:00:00+00:00 to 2025-06-04 12:00:00+00:00
firewx_stns_current.fid--4699bdee_1973debb200_-48a7: from 2025-06-04 12:00:00+00:00 to 2025-06-04 12:00:00+00:00
firewx_stns_current.fid--4699bdee_1973debb200_-48a6: from 2025-06-04 12:00:00+00:00 to 2025-06-0